In [ ]:
# !pip install xarray matplotlib cartopy xemsf netCDF4 

In [ ]:
# !pip install OWSLib  # for Blue Marble access
# !pip install vsrife

In [ ]:
# !mkdir /Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/frames2

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from pathlib import Path
import os
from subprocess import run
from matplotlib.colors import LinearSegmentedColormap, to_rgba
import matplotlib.colors as mcolors
from matplotlib.image import imread

In [ ]:
# Read in huge Blue Marble image 
# --- Place this code BEFORE your 1000-frame loop (Load ONCE) ---

# Disable the decompression bomb check
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

# --- Configuration (MUST match your image and desired georeference) ---
STATIC_MAP_FILE = '/Users/bmapes/Box/Sky_Symphony_Box/BlueMarble.200408.3x21600x10800.jpg' 
FULL_WORLD_EXTENT = [-180, 180, -90, 90] 
try:
    # Load the single satellite image file ONLY ONCE
    BASE_IMAGE_DATA = imread(STATIC_MAP_FILE)
except FileNotFoundError:
    raise FileNotFoundError(f"Error: Static map file '{STATIC_MAP_FILE}' not found. Please download a high-res Blue Marble image and save it to this name.")

In [ ]:
# ==== USER CONFIG ====  # frames3 has latest color scale, alpha never 0 
data_folder = Path("/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/ncfiles/") 
frame_folder =Path("/Volumes/Samsung USB/frames_bluemarble/")
frame_folder.mkdir(exist_ok=True)

dpi = 150
# =====================

frame_folder.mkdir(exist_ok=True)

# 2. Preload lat/lon grids from all files
nc_files = sorted(data_folder.glob("*.nc"))
files_info = []
for file in nc_files:
    with xr.open_dataset(file) as ds:
        files_info.append({
            "file": file,
            "lat": ds['lat'].load(),
            "lon": ds['lon'].load()
        })

print('Got all files_info (lat-lon for interpolations)')

In [ ]:
ds_all = xr.open_mfdataset(nc_files)
ds_all

In [ ]:
# Custom color scale, including transparency to see Blue Marble underneath  
color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue (color enhancement ends)
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.9),  # white semitrans light gray
    (290, "#ffffff", 0.6),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.4),  # bush green cool surface --> WHITE
    (310, "#ff8000", 1.0),  # HOT surface orange, see https://html-color.codes/
    (340, "#000000", 1.0)  # HOTHOT surface black
]

# Brigher colors from BlueMarble (less white haze at high T), less garish hot orange
color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue (color enhancement ends)
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.7),  # white semitrans light gray
    (290, "#ffffff", 0.3),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.3),  # --> WHITE 
    (310, "#ff8000", 0.6),  # HOT surface orange, see https://html-color.codes/
    (340, "#000000", 0.8)  # HOTHOT surface black
]

vmin = 190; vmax=340 

cmap = LinearSegmentedColormap.from_list("custom", [
    ((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points
])

In [ ]:
# Find place to RESTART, based on existing image frame filenames 
existing_frames = [
    int(f.stem) for f in frame_folder.glob("*.png")
]
frame_count = max(existing_frames) if existing_frames else -1
print(f'Resuming from frame {frame_count + 1}')

done_frame_count = frame_count//2 *2  # Remake the last even number that exists, to catch the odd missing one
frame_count = done_frame_count
done_frame_count

In [ ]:
# 3. Starting from the point above, Generate plots with smoothed coastline movement, 
# RESTART after existing file names if it crashes 

for i, file in enumerate(files_info[done_frame_count//2:]):
    ds = xr.open_dataset(file['file'])
    Tb_all = ds['Tb']

# Interpolate the coastline and blue marble extent for half-hourly files. 
# CAREFUL! Xarray does this wrong, work with scalar min and max values.
    # Current file (i:00)
    lat_now = file['lat']
    lon_now = file['lon']
    
    if i < len(files_info) - 1:
        # Next file's lat-lon info ((i+1):00)
        lat_next = files_info[done_frame_count//2+i+1]['lat']
        lon_next = files_info[done_frame_count//2+i+1]['lon']
    else:
        # Use current file if it's the last file
        lat_next = lat_now
        lon_next = lon_now

    
# Loop over the 2 times in each file, interpolating the extent of the plot
    for t in range(Tb_all.sizes['time']):
        Tb = Tb_all.isel(time=t)
        timestamp = str(Tb_all.time.values[t])[:16].replace(":", "").replace("T", "_")

        # Smooth coastlines for 2nd timestep
        if t == 0:
            coast_latmin = min(lat_now)
            coast_lonmin = min(lon_now)
            coast_latmax = max(lat_now)
            coast_lonmax = max(lon_now)
        else:
            coast_latmin = 0.5 * (min(lat_now) + min(lat_next))
            coast_lonmin = 0.5 * (min(lon_now) + min(lon_next))
            coast_latmax = 0.5 * (max(lat_now) + max(lat_next))
            coast_lonmax = 0.5 * (max(lon_now) + max(lon_next))

        # Define extent from coast file
        extent = [
            float(coast_lonmin), float(coast_lonmax),
            float(coast_latmin), float(coast_latmax)
        ]
        

# Plot data on top of map image, there is some transparency 
        fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
        ax.set_extent(extent, crs=ccrs.PlateCarree())
        
# Background image: stock image, or slow web call, tiles keeps crashing on me, sigh
        #ax.stock_img()  # Adds low-res natural Earth background
        #ax.add_wmts('https://map1c.vis.earthdata.nasa.gov/wmts-geo/wmts.cgi', 
        #            'BlueMarble_NextGeneration')
        ax.imshow(BASE_IMAGE_DATA,
          origin='upper',
          transform=ccrs.PlateCarree(),
          extent=FULL_WORLD_EXTENT)

        ax.coastlines()
        #ax.coastlines(resolution='50m', color='white', linewidth=0.5, zorder=3)
        #ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.set_title(f"Tb at {timestamp}", fontsize=12)

        Tb.interpolate_na(dim="lon", method="linear", max_gap=2).plot.imshow(
            ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin,vmax=vmax,\
            add_colorbar=False, #cbar_kwargs={'label': 'Brightness Temperature [K]'}
        )
        plt.tight_layout()

        out_file = frame_folder / f"{frame_count:08d}.png"

        plt.savefig(out_file, dpi=dpi)
        plt.close()
        
        print(f"Saved {out_file.name}")
        frame_count += 1
        #print('starting on frame_count ',frame_count) 


In [ ]:
# Crop off the edges 

file = '/Volumes/Samsung USB/frames_bluemarble//00000926.png'
frame_folder =Path("/Volumes/Samsung USB/frames_bluemarble/")

from PIL import Image

pic = Image.open(file)
width, height = pic.size
# pic.crop( (210,70,1290,850) )  #.size 


#pic.size #(1500, 900)
#w, h, w, h and first height is down from top 
# pic.crop( (250,100,1250,800) )  #.size  # (1000, 700)
#pic.crop( (210,70,1290,850) )  #.size I like this one 

In [ ]:
# Crop them all 

from pathlib import Path; from PIL import Image
SOURCE = Path('/Volumes/Samsung USB/frames_bluemarble/')
OUTPUT = Path('/Volumes/Samsung USB/frames_bluemarble_crop/')
OUTPUT.mkdir(exist_ok=True) 

CROP = (210, 70, 1290, 850)
for f in SOURCE.glob('0000*.png'): Image.open(f).crop(CROP).save(OUTPUT / f.name)

In [ ]:
# 4. Create simple video using ffmpeg

video_filename = "/Volumes/Samsung USB/"+"bluemarble_cropped.mp4"

ffmpeg_cmd = [
    "ffmpeg", "-y", "-framerate", "6",
    "-i", str(frame_folder / "%08d.png"),
    "-c:v", "libx264", "-pix_fmt", "yuv420p",
    video_filename
]

print("Creating video...")
run(ffmpeg_cmd)
print(f"✅ Video saved as {video_filename}")

### Claude's ffmpeg approach to interplating to 8 minutes length 

#Interpolation factor × 1400 frames ÷ target seconds = needed fps
#3x: 4200 frames / 480s = 8.75 fps (8 min) or 7 fps (10 min)
#4x: 5600 frames / 480s = 11.67 fps (8 min) or 9.33 fps (10 min)

##### First create base video
#ffmpeg -y -framerate 6 -i frame_%04d.png -c:v libx264 -pix_fmt yuv420p video_filename

#### Then interpolate and slow down to 8 minutes
##### Target: 480 seconds from 233 seconds = 2.06x slower
##### ffmpeg -i video_filename -filter:v "minterpolate='fps=24',setpts=2.06*PTS" video_filename+'8min.mp4'

In [ ]:
# Interpolate -->frames8 using rife, 

import subprocess

frame_folder   = Path('/Volumes/Samsung USB/frames_bluemarble_crop')
framess_folder = Path('/Volumes/Samsung USB/frames_bluemarble_crop_8x')
framess_folder.mkdir(exist_ok=True)

# Run RIFE interpolation to make interleaved frames 
subprocess.run([
    "/Volumes/Samsung USB/rife-ncnn-vulkan-20221029-macos/rife-ncnn-vulkan",  # or full path to executable
    "-i", frame_folder,
    "-o", framess_folder,
    "-n", "10640",  # number of frames to generate, 8x (1330)
    "-m", "rife-v4.6"
])

In [ ]:
# Then from interpolated frames make video at desired length
# video_filename = "/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/"+"Irma_bmc_4xframes.mp4"
video_filename = "/Volumes/Samsung USB/"+"bluemarble_cropped_interp8x.mp4"

subprocess.run([
    "ffmpeg", "-y",
    "-framerate", "24",  # 11.67 = 5600 frames / 480 sec for 8 min, I have 5328 frames 
    "-i", f"{framess_folder}/%08d.png",
    "-c:v", "libx264", "-pix_fmt", "yuv420p",
    "/Volumes/Samsung USB/Irma_bmc_8xframes_framerate24.mp4"
])

In [ ]:
!open /Volumes/Samsung USB/Irma_bmc_8xframes_framerate24.mp4

In [ ]:
# Custom color scale, including transparency to see Blue Marble underneath  
# THIS ONE WAS A BIT GAUZY, MAKE MORE TRANSPARENCY AT LOW LEVELS / HIGH TB

color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue (color enhancement ends)
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.9),  # white semitrans light gray
    (290, "#ffffff", 0.6),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.4),  # bush green cool surface --> WHITE
    (310, "#ff8000", 1.0),  # HOT surface orange, see https://html-color.codes/
    (340, "#000000", 1.0)  # HOTHOT surface black
]

vmin = 190; vmax=340 

cmap = LinearSegmentedColormap.from_list("custom", [
    ((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points
])

In [ ]:
# Test colors for one image 

data_folder = Path("/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/ncfiles/")  
nc_files = sorted(data_folder.glob("*.nc"))

In [ ]:
ds = xr.open_dataset(nc_files[200])  # 21Z = 18+24-15 cold 
ds = xr.open_dataset(nc_files[200-12 +24*0])      # Sahel coast crossing 

Tb = ds.Tb[0]

# Define extent from coast file
extent = [ds.lon.min().values, ds.lon.max().values,ds.lat.min().values, ds.lat.max().values]

# Plot data on top of map image, there is some transparency 
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent(extent, crs=ccrs.PlateCarree())

# Background image: stock image, or slow web call, tiles keeps crashing on me, sigh
ax.imshow(BASE_IMAGE_DATA,
  origin='upper',
  transform=ccrs.PlateCarree(),
  extent=FULL_WORLD_EXTENT)

ax.coastlines()

Tb.interpolate_na(dim="lon", method="linear", max_gap=2).plot.imshow(
    ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin,vmax=vmax,\
    add_colorbar=True, #cbar_kwargs={'label': 'Brightness Temperature [K]'}
)
plt.tight_layout()


In [ ]:
# Custom color scale, including transparency to see Blue Marble underneath  
color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue (color enhancement ends)
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.7),  # white semitrans light gray
    (290, "#ffffff", 0.3),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.3),  # --> WHITE 
    (310, "#ff8000", 0.6),  # HOT surface orange, see https://html-color.codes/
    (340, "#000000", 0.8)  # HOTHOT surface black
]

vmin = 190; vmax=340 
cmap = LinearSegmentedColormap.from_list("custom", [
    ((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points
])

ds = xr.open_dataset(nc_files[200-12 +24*17 +8])  # End of movie afternoon 
ds = xr.open_dataset(nc_files[200-12 +24*0])      # Sahel coast crossing 

Tb = ds.Tb[0]
# Define extent from datafile
extent = [ds.lon.min().values, ds.lon.max().values,ds.lat.min().values, ds.lat.max().values]


# Plot data on top of map image, there is some transparency 
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent(extent, crs=ccrs.PlateCarree())

# Background image: stock image, or slow web call, tiles keeps crashing on me, sigh
ax.imshow(BASE_IMAGE_DATA,
  origin='upper',
  transform=ccrs.PlateCarree(),
  extent=FULL_WORLD_EXTENT)

ax.coastlines()

Tb.interpolate_na(dim="lon", method="linear", max_gap=2).plot.imshow(
    ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin,vmax=vmax,\
    add_colorbar=True, #cbar_kwargs={'label': 'Brightness Temperature [K]'} 
);
plt.tight_layout()